# DSN AI Bootcamp 2025 Qualification Hackaton  
This is the machine learning with python track 
we have been given a dataset that was genereated by a deep learning AI trained on the used Car Price Prediction dataset. check it out [Here](https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset/data) 

Our goal is to build a machine learning model that can learn from the information of the past use of the vehicles , so that we can predict the price that the vehicle will be sold at.


- in this notebook we are going to build a catboost model that will predict on both categorical and numeric dataset 


In [29]:
# Importing the Needed Libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import textwrap 
from ast import Break

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.ensemble import VotingRegressor

from sklearn.model_selection import GridSearchCV

import optuna 
import catboost as cat 
from catboost import Pool
import xgboost as xgb 
import lightgbm as lgb 

In [30]:
# importing the training and testing dataset into the notebook 

train_df = pd.read_csv("/kaggle/input/hackathon-qualification/archive/train.csv")

test_df = pd.read_csv("/kaggle/input/hackathon-qualification/archive/test.csv")

## Exploratory Data Analysis 
Now to explore the data
here are some basic informaton on the dataset columns. 

The train_data is a dataset with 188533 rows(entries) and 13 columns (features).
while The test_data is a dataset with 125690 rows(entries) and 12 columns (features).

> **columns info**:    
>  **Brand & Model** : Identify the brand or company name along with the specific model of each vehicle.    
>  **Model Year** : Discover the manufacturing year of the vehicles, crucial for assessing depreciation and technology advancements.    
> **Mileage** : Obtain the mileage of each vehicle, a key indicator of wear and tear and potential maintenance requirements.    
> **Fuel Type** : Learn about the type of fuel the vehicles run on, whether it's gasoline, diesel, electric, or hybrid.     
> **Engine Type** : Understand the engine specifications, shedding light on performance and efficiency.    
> **Transmission** : Determine the transmission type, whether automatic, manual, or another variant.    
> **Exterior & Interior Colors** : Explore the aesthetic aspects of the vehicles, including exterior and interior color options.    
> **Accident History** : Discover whether a vehicle has a prior history of accidents or damage, crucial for informed decision-making.    
> **Clean Title**: Evaluate the availability of a clean title, which can impact the vehicle's resale value and legal status.     
> **Price** : Access the listed prices for each vehicle, aiding in price comparison and budgeting.


In [55]:
# checking out the informaion of the data set 
"""
This .info() is quite important has it gives us quick information
about the datasset like the shape, the columns names and so on.
"""
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


## Ploting the pairplot of the data 
 pairplot gives an overall overview of the dataset (it lets you sneak a peek at the dataset )

> **Note:**
> it only plots out numerical columns  

In [ ]:
sns.pairplot(train_df)

## Ploting the values count of all the columns in the train_df 
- This helps me check the amount of unique values in each column.
- this can also help when trying to figure out which column to perform feature engineering on
- The function below helps to plot the value count(sum of unique values) of columns

In [ ]:
# function to plot int value_counts

def plot_numeric_value_counts(column,dataframe):
    """
    this function plots the value count of numeric columns.
    """
    print(f"Ploting graph for {column}")
    
    index_val = dataframe[column].value_counts().index
    value_val = dataframe[column].value_counts().values
    
    plt.bar(index_val, value_val)
    plt.xlabel(column)
    plt.ylabel('count')
    plt.title(f'{column} value count')
    plt.show()

# function to plot object value count
def plot_object_value_counts(column,dataframe):
    """
    this function plots the value count of categorical columns.
    it uses textwrap to split the strings for aa more readable graph.
    """
    print(f"Ploting graph for {column}")
    index_counts = dataframe[column].value_counts()
    index_val = [textwrap.fill(label, 7) for label in index_counts.index]
    value_val = dataframe[column].value_counts().values
    
    plt.bar(index_val, value_val)
    
    plt.xlabel(column)
    plt.ylabel('count')
    plt.title(f'{column} value count')
    
    plt.show()


# This is the main function
def plot_valuecounts(dataframe):
    """
    this function checks if a column is numeric or catergrical 
    if so it runs the select function("plot_numeric_value_counts" or "plot_object_value_counts") on the column
    """
    for columns in dataframe.columns:
        if pd.api.types.is_numeric_dtype(dataframe[columns]):
          # function to plot int or floatvalue count
          plot_numeric_value_counts(columns,dataframe)
        elif pd.api.types.is_object_dtype(dataframe[columns]):
          # function to plot object value count
          plot_object_value_counts(columns,dataframe)
        else:
          Break


In [ ]:
# The id column has too much unique column (188533 entries) so i had to drop it to save computing power and time
train_df_values = train_df.drop(columns = "id", axis = 1)

plot_valuecounts(train_df_values)

## Insights
- The Clean_title column only has one value so when cleaning we will drop the column it
- there are outlier data in our model_year column (the cars made before 1980)
- Note: that most of the vehicles that we have are of fuel_type gasoline
- so the fuel_type, Accident columns are imbalanced
- we will have to use stratified sampling when training the model
- the Engine has many places for feature engineering
    - the format "355.0HP  5.3L 8 Cylinder Engine Gasoline Fuel" can be 
    - (355.0HP)the horsepower
    - (5.3L) the litres of fuel
    - (8 Cylinder)the cylinder
    - (Gasoline Fuel) the type of engine 

## Checking the mileage to the price ratio with respect to model year 
i just wanted to get a look into the dataset and see if there is any relationship between the milage and the price column with respect to the model_year columns.
it answers questions like:
- are cars with higher milage more expensive ?
- are cars with lower milage and lower model_year(old cars) more expensive or cheap ?

#### My insights 
- cars will very much milage sell for very low prices
- some old cars will sell for high price
- newer cars obviously have lower milage

- Overall insight: i am not sure any of this features are that important(am not saying that they are not important)

In [ ]:
colors = np.random.rand(len(train_df)) * 100

scatter = plt.scatter(train_df['milage'],train_df["price"], c=train_df["model_year"], cmap='viridis', s=100, alpha=0.7)
plt.colorbar(scatter, label='Model Year')
plt.xlabel("milage")
plt.ylabel("Price")
plt.title("Graph of Milage against Price with reference to the Model Year")
plt.show()

## Cleaning the data 

"I would like to credit Mart Preusse [Notebook](https://www.kaggle.com/code/martinapreusse/ps4e9-cat-svr-lgbm-nn-py#INTRODUCTION) for his feature engineering process."

In [31]:
def feat_eng(data):
    """
    perfroms feature Engineering on the dataset.
    """
    #categorical features
    cat_cols = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 
                'int_col', 'accident']
    df = data.copy()
    for feature in ['fuel_type', 'accident', 'clean_title']:
        df[feature] = df[feature].fillna("no_entry")
    df['clean_title'] = np.where(df['clean_title']== 'Yes',1,0).astype('int16')
    df['fuel_type'] = df['fuel_type'].replace({'–':'not supported'})
    df['mileage'] = df['milage']//100
    for feature in cat_cols:
        df[feature] = df[feature].astype(object)
    #numerical features
    df['horsepower'] = df['engine'].str.extract(r'(\d+\.?\d*)HP').astype('float').fillna(-1)
    df['displacement'] = df['engine'].str.extract(r'(\d+(\.\d+)?)L')[0].astype(float).fillna(-1)
    df['age'] = 2025 - df['model_year']
    
    return df

In [8]:
def check_cat_cols(dataframe):
    """
    checks for the categorical columns in dataframe and adds to a list 
    """
    cat_list = []
    for columns in dataframe:
        if dataframe[columns].dtype == "object":
            cat_list.append(columns)
    return cat_list

In [56]:
#calling the feat_eng function on the train_df
train_df1 = feat_eng(train_df)

In [57]:
# calling the check_cat_cols function on the resulting function 
cat_list = check_cat_cols(train_df1)

In [58]:
# droping the target column and id column in the training data 
X_train_cat = train_df1.drop(['price', 'id'], axis = 1)

y_train_cat = train_df1['price']

In [59]:
#performing feature engineering on the test data 
test_df1 = feat_eng(test_df)

In [60]:
#droping the id column in the test data 
the_id = test_df1['id']

X_test_cat = test_df1.drop([ 'id'], axis = 1)

## Training the model 

In [61]:
cat_model = cat.CatBoostRegressor(n_estimators=363, learning_rate=0.048,max_depth = 7,cat_features= cat_list, random_state=42, verbose=0)
cat_model.fit(X_train_cat, y_train_cat)

In [62]:
# predicting the test data
cat_preds = cat_model.predict(X_test_cat)

In [40]:
# Function to convert our prediction to csv format 
def to_csv(dataframe,name):
    dataframe = dataframe.round(3)    
    submit_data = pd.DataFrame({"id":the_id.values,
                  "price":dataframe})
    name = name
    print(submit_data.head())
    submit_data.to_csv(f"{name}.csv", index=False)

In [63]:
to_csv(cat_preds,"cat_submission")

       id      price
0  188533  16230.075
1  188534  84080.582
2  188535  56152.847
3  188536  34951.548
4  188537  26889.621
